# ReadMe

This notebook is to connect and experiment with materials project [api](https://api.materialsproject.org/docs#/Materials%20Thermo/search_materials_thermo_phase_diagram__get)

In [30]:
import chainlit as cl
from langchain.llms import OpenAI
from langchain.chains import LLMChain, APIChain
from langchain.memory.buffer import ConversationBufferMemory
from dotenv import load_dotenv

load_dotenv()

True

In [31]:
from typing import Any, Dict, List, Optional, Sequence, Tuple
from urllib.parse import urlparse

from langchain_community.utilities.requests import TextRequestsWrapper
from langchain_core.callbacks import (
    AsyncCallbackManagerForChainRun,
    CallbackManagerForChainRun,
)
from langchain_core.language_models import BaseLanguageModel
from langchain_core.prompts import BasePromptTemplate
from langchain_core.pydantic_v1 import Field, root_validator

from langchain.chains.api.prompt import API_RESPONSE_PROMPT, API_URL_PROMPT
from langchain.chains.base import Chain
from langchain.chains.llm import LLMChain

In [33]:
from api_docs.materials_project_doc import material_api_docs
from langchain.prompts import PromptTemplate

In [85]:
api_url_template = """
Given the following API Documentation for Material Project's
API: {api_docs}
Your task is to construct the most efficient API URL to answer 
the user's question, ensuring the 
call is optimized to include only necessary information.
Question: {question} with license="BY-C" and _limit=10
API URL:
"""
api_url_prompt = PromptTemplate(input_variables=['api_docs', 'question'],
                                template=api_url_template)


In [94]:
api_response_template = """"
With the API Documentation for Material Platform's official API: {api_docs} 
and the user question: {question} in mind,
and given this API URL: {api_url} for querying, here is the 
response from Material Project's API: {api_response}. 
Please provide a summary that directly addresses the user's question based on the response, 
omitting technical details like response format, and 
focusing on delivering the answer with clarity and conciseness, 
as if a material scientist is providing this information.
Summary:
"""

api_response_prompt = PromptTemplate(input_variables=['api_docs',
                                                      'question', 
                                                      'api_url',
                                                      'api_response'],
                                     template=api_response_template)

In [95]:
llm = OpenAI(model='gpt-3.5-turbo-instruct',
             temperature=0)

In [96]:
api_chain = APIChain.from_llm_and_api_docs(
        headers={'X-API-KEY': 'cHlaLNDodbKlfGkbYUQwCkOzE6GK8zdD'},
        llm=llm,
        api_docs=material_api_docs,
        api_url_prompt=api_url_prompt,
        api_response_prompt=api_response_prompt,
        verbose=True,
        limit_to_domains=["https://api.materialsproject.org/materials/core/"]
    )

In [97]:
api_chain.run("List me everything relevant to TiO2")



> Entering new APIChain chain...
https://api.materialsproject.org/materials/core/?formula=TiO2&license=BY-C&_limit=10
{"data":[{"formula_pretty":"TiO2","material_id":"mp-572822","last_updated":"2021-02-15T21:42:53.920000"},{"formula_pretty":"TiO2","material_id":"mp-1245134","last_updated":"2021-02-06T06:45:27.105000"},{"formula_pretty":"TiO2","material_id":"mp-1350429","last_updated":"2021-02-10T12:13:15.751000"},{"formula_pretty":"TiO2","material_id":"mp-777129","last_updated":"2021-02-16T19:01:11.722000"},{"formula_pretty":"TiO2","material_id":"mp-554278","last_updated":"2023-10-08T13:05:51.760000"},{"formula_pretty":"TiO2","material_id":"mp-1245321","last_updated":"2021-02-05T12:43:10.024000"},{"formula_pretty":"TiO2","material_id":"mp-1245308","last_updated":"2021-02-05T14:50:26.120000"},{"formula_pretty":"TiO2","material_id":"mp-775938","last_updated":"2021-03-06T00:24:01.692000"},{"formula_pretty":"TiO2","material_id":"mp-1840","last_updated":"2023-10-07T06:05:01.300000"},{"for

"\nThe Material Project's API provides a list of 10 relevant materials for TiO2, including their material IDs, formula, and last updated date. These materials have a license of BY-C, indicating they are available for non-commercial use. The API also includes information on the total number of materials available for TiO2 and the maximum limit for results."